# Imports

In [ ]:
pip install --upgrade keras-cv

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/AN2DL

In [ ]:
# Set seed for reproducibility
seed = 42

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(seed)
random.seed(seed)

# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import mixed_precision
#mixed_precision.set_global_policy('mixed_float16')
print(f"TensorFlow version {tf.__version__}")
print(f"Keras version {tfk.__version__}")


# Set seed for TensorFlow
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Reduce TensorFlow verbosity
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import seaborn as sns
from PIL import Image
import matplotlib.gridspec as gridspec
import keras_cv as kcv
from keras_cv.layers import RandomApply
from keras.optimizers import Lion

TensorFlow version 2.17.1
Keras version 3.5.0


# Dataset Loading


In [5]:
data = np.load('preprocessed.npz', allow_pickle=True)
X_train = data['train_images']
y_train = data['train_labels']
X_val = data['val_images']
y_val = data['val_labels']

# Augmentation

In [ ]:
# Data augmentation pipeline
data_augmentation = tfk.Sequential([
    RandomApply(kcv.layers.JitteredResize((96, 96), (0.9, 1)), 0.7),
    RandomApply(kcv.layers.RandomTranslation(0.5, 0.5), 0.6),
    RandomApply(kcv.layers.RandAugment((0, 255)), 0.8),
    RandomApply(kcv.layers.RandomRotation(1), 0.6),
    RandomApply(kcv.layers.RandomContrast((0, 255), 0.7), 0.4),
    RandomApply(kcv.layers.RandomBrightness(0.1, value_range=(0.0, 255.0)), 0.4),
    RandomApply(kcv.layers.RandomZoom(height_factor=(-0.1, 0), fill_mode="reflect"), 0.4),
])

# Dataset creation
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=1024).batch(64).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64).prefetch(tf.data.AUTOTUNE)

# Augmentation is applied to training dataset
train_dataset_augmented = train_dataset.map(
    lambda x, y: (data_augmentation(x), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Transfer Learning

In [ ]:
def transfer_learning():
    # ConvNeXtXLarge is used as Base model
    base_model = tfk.applications.ConvNeXtXLarge(weights='imagenet', include_top=False, include_preprocessing=True, input_shape=(96, 96, 3), pooling='avg', name='convnext_xlarge')

    for layer in base_model.layers:
        layer.trainable = False

    inputs = tfk.Input(shape=(96, 96, 3))
    x = base_model(inputs, training=False)

    x = tfkl.Dense(1024, kernel_initializer='he_normal')(x)
    x = tfkl.Activation('relu')(x)
    x = tfkl.Dropout(0.4)(x)

    # Classifier 8 classes
    outputs = tfkl.Dense(8, activation='softmax', kernel_initializer=tfk.initializers.GlorotNormal(seed=seed))(x)
    model = tfk.Model(inputs, outputs)

    optimizer  = Lion(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model, base_model

model, base_model = transfer_learning()

### Fitting Transfer Learning

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=5, restore_best_weights=True)
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
checkpoint = tfk.callbacks.ModelCheckpoint("best_model.keras", monitor="val_accuracy", save_best_only=True, mode="max")

history = model.fit(
          train_dataset_augmented,
          validation_data=val_dataset,
          epochs=40,
          batch_size=64,
          callbacks =[early_stopping, lr_scheduler, checkpoint],
).history

Epoch 1/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 278s 1s/step - accuracy: 0.8399 - loss: 0.4684 - val_accuracy: 0.9674 - val_loss: 0.0985 - learning_rate: 1.0000e-05
Epoch 2/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.8300 - loss: 0.4801 - val_accuracy: 0.9691 - val_loss: 0.1015 - learning_rate: 1.0000e-05
Epoch 3/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.8446 - loss: 0.4687 - val_accuracy: 0.9666 - val_loss: 0.1024 - learning_rate: 1.0000e-05
Epoch 4/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - accuracy: 0.8405 - loss: 0.4611 - val_accuracy: 0.9691 - val_loss: 0.0984 - learning_rate: 1.0000e-05
Epoch 5/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.8463 - loss: 0.4522 - val_accuracy: 0.9624 - val_loss: 0.1037 - learning_rate: 1.0000e-05
Epoch 6/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 202s 1s/step - accuracy: 0.8386 - loss: 0.4628 - val_accuracy: 0.9699 - val_loss: 0.0958 - learning_rate: 1.0000e-05
Epoch 7/40
169/169 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - acc

KeyboardInterrupt: 

In [ ]:
model.save("TLConvNeXtXLarge.keras", include_optimizer=True)

# Fine Tuning

###### The model is fine-tuned in two phases: first by unfreezing the last half of the layers to train higher-level features, and then by unfreezing all layers to fine-tune the entire model. Each phase uses a progressively smaller learning rate to ensure stability during training.

In [ ]:
def fine_tuning(model, base_model, train_since=137):
    for layer in base_model.layers[:train_since]:
        layer.trainable = False

    for layer in base_model.layers[train_since:]:
        if not isinstance(layer, tfkl.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

    for i, layer in enumerate(base_model.layers):
        print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")


    optimizer = Lion(learning_rate=1e-6, gradient_accumulation_steps=2)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = fine_tuning(model, base_model, train_since=137)

###### Gradient accumulation is used to simulate larger batch sizes, so that gradient is accumulated over 2 mini-batches

In [ ]:
def fine_tuning(model, base_model, train_since=0):
    for layer in base_model.layers[:train_since]:
        layer.trainable = False

    for layer in base_model.layers[train_since:]:
        if not isinstance(layer, tfkl.BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False

    for i, layer in enumerate(base_model.layers):
        print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

    optimizer = Lion(learning_rate=1e-6, gradient_accumulation_steps=2)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = fine_tuning(model, base_model, train_since=0)

### Model Loading

In [14]:
model = tfk.models.load_model('FTConvNeXtXLarge.keras')
base_model = model.get_layer("convnext_xlarge")

In [15]:
optimizer = Lion(learning_rate=1e-6, gradient_accumulation_steps=2)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Fitting Fine Tuning

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-8)
checkpoint = tfk.callbacks.ModelCheckpoint("best_model.keras", monitor="accuracy", save_best_only=True, mode="max")

history = model.fit(
          train_dataset_augmented,
          validation_data=val_dataset,
          epochs=5,
          batch_size=64,
          callbacks =[early_stopping, lr_scheduler, checkpoint],
).history

# Calculate the final validation accuracy
final_val_accuracy = round(history['val_accuracy'][-1] * 100, 2)

Epoch 1/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 486s 2s/step - accuracy: 0.9485 - loss: 0.1459 - val_accuracy: 0.9891 - val_loss: 0.0332 - learning_rate: 1.0000e-06
Epoch 2/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.9508 - loss: 0.1449 - val_accuracy: 0.9883 - val_loss: 0.0337 - learning_rate: 1.0000e-06
Epoch 3/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.9450 - loss: 0.1511 - val_accuracy: 0.9866 - val_loss: 0.0340 - learning_rate: 1.0000e-06
Epoch 4/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 354s 2s/step - accuracy: 0.9536 - loss: 0.1388 - val_accuracy: 0.9866 - val_loss: 0.0335 - learning_rate: 1.0000e-06
Epoch 5/5
169/169 ━━━━━━━━━━━━━━━━━━━━ 386s 2s/step - accuracy: 0.9510 - loss: 0.1466 - val_accuracy: 0.9883 - val_loss: 0.0336 - learning_rate: 5.0000e-07


# Save Model

### Save Model for more Training

In [ ]:
model.save("FTConvNeXtXLarge.keras", include_optimizer=False)

### Save Final Model

In [10]:
model.training = False
model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(), optimizer=None, metrics=["accuracy"])
model.save("ConvNeXtLarge.keras", include_optimizer=False)

# Load File .keras

In [11]:
%%writefile model.py
import numpy as np

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl

class Model:
    def __init__(self):
        """
        Initialize the internal state of the model. Note that the __init__
        method cannot accept any arguments.

        The following is an example loading the weights of a pre-trained
        model.
        """
        self.neural_network = tfk.models.load_model('ConvNeXtLarge.keras', compile=False)
        self.neural_network.compile(loss=tfk.losses.SparseCategoricalCrossentropy(), optimizer=None, metrics=["accuracy"])

    def predict(self, X):
        """
        Predict the labels corresponding to the input X. Note that X is a numpy
        array of shape (n_samples, 96, 96, 3) and the output should be a numpy
        array of shape (n_samples,). Therefore, outputs must no be one-hot
        encoded.

        The following is an example of a prediction from the pre-trained model
        loaded in the __init__ method.
        """
        preds = self.neural_network.predict(X)
        if len(preds.shape) == 2:
            preds = np.argmax(preds, axis=1)
        return preds

Writing model.py


In [12]:
from datetime import datetime
filename = f'submission_{datetime.now().strftime("%y%m%d_%H%M%S")}.zip'

# Add files to the zip command if needed
!zip {filename} model.py ConvNeXtLarge.keras

  adding: model.py (deflated 54%)
  adding: ConvNeXtLarge.keras (deflated 7%)
